In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
import torch.optim as optim
from torchvision.transforms import ToTensor
import numpy as np
import matplotlib.pyplot as mp
import pandas as ps


In [ ]:
class data(Dataset):
    def __init__(self,texts,images):
        self.texts=texts
        self.images=images

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text=self.texts[idx]
        image=self.images[idx]
        return text,image


class TE(nn.Module):
    def __init__(self,input_dim,model_dim):
        super(TE,self).__init__()
        self.encoder=nn.TransformerEncoderLayer(d_model=model_dim,nhead=8)
        self.embedding=nn.Embedding(input_dim,model_dim)
        #self.gelu=nn.GELU()

    def forward(self,x):
        x=self.embedding(x)
        output=self.encoder(x)
        return output  #self.gelu(output)


class ID(nn.Module):
    def __init__(self,image_size,model_dim):
        super(ID,self).__init__()
        self.decoder=nn.Linear(model_dim,image_size)
        #self.relu=nn.ReLU()

    
    def forward(self,x):
        output=self.decoder(x)
        return output #self.relu(output)



In [ ]:
input_size=256# 예: 텍스트 토큰의 수
model_dim=512# 모델의 차원
image=64*64*3# 예: 생성할 이미지의 크기 (64x64 RGB)

encoder=TE(input_size,model_dim)
decoder=ID(image_size,model_dim)

params=list(encoder.parameters())+list(decoder.parameters())
opt=optim.SGD(params,lr=1e-4)

text=torch.randint(0,input_size,(100,10))
print(type(text))
image=torch.rand(100,image_size)
print(type(image))

dataset=data(text,image)

print(type(dataset))

dataloader=DataLoader(dataset,batch_size=10,shuffle=True)

print(type(dataloader))


In [ ]:
num_epochs=100
num_epochs = 100
for epoch in range(num_epochs):
    for text_batch, image_batch in dataloader:  # 변수 이름 변경
        encoded = encoder(text_batch)  # 객체 이름과 함수 호출을 구분
        decoded = decoder(encoded.mean(dim=1))  # 출력을 decoded 변수에 할당
        loss = nn.MSELoss()(decoded, image_batch)  # 올바른 변수 이름 사용
        opt.zero_grad()
        loss.backward()
        opt.step()
    print(f'Epoch {epoch}, Loss: {loss.item()}')

total_loss = 0
criter = nn.MSELoss()
encoder.eval()  # 모델을 평가 모드로 설정
decoder.eval()
with torch.no_grad():  # 기울기 계산 비활성화
    for text_batch, image_batch in dataloader:
        encoded = encoder(text_batch)
        decoded = decoder(encoded.mean(dim=1))
        loss = criter(decoded, image_batch)
        total_loss += loss.item()
avg_loss = total_loss / len(dataloader)

print(f'Test Loss: {avg_loss}')  # 수정된 부분
 

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TG(nn.Module):
    def __init__(self,vocab,d_model,nhead,num_layer1,num_layer2,dim_ff, max_len):
        super(TG,self).__init__()
        self.embedding=nn.Embedding(vocab,d_model)
        self.pos_encode=nn.Parameter(torch.randn(1,max_len,d_model))
        self.transformer=nn.Transformer(d_model=d_model,nhead=nhead,num_encoder_layers=num_layer1,num_decoder_layers=num_layer2, dim_feedforward=dim_ff)
        self.fc=nn.Linear(d_model,vocab)
        self.d_model=d_model

    def forward(self,x,y):
        x=self.embedding(x)*torch.sqrt(torch.tensor(self.d_model,dtype=torch.float32))
        y=self.embedding(y)*torch.sqrt(torch.tensor(self.d_model,dtype=torch.float32))
        y+=self.pos_encode[:,:y.size(1),:]
        x+=self.pos_encode[:,:x.size(1),:]
        output=self.transformer(x,y)
        output=self.fc(output)
        return output

# 모델 파라미터 설정
vocab = 10000  # 예시로 사용된 어휘 크기
d_model = 512  # 임베딩 차원
nhead = 8  # 멀티헤드 어텐션의 헤드 수
num_layer1= 4  # 인코더 레이어 수
num_layer2= 4  # 디코더 레이어 수
dim_ff= 2048  # 피드포워드 네트워크의 차원
max_len= 100  # 최대 시퀀스 길이

device=torch.device('cuda'if torch.cuda.is_available()else'cpu')
print(f'{device} mode')
TG=TG(vocab,d_model,nhead,num_layer1,num_layer2,dim_ff, max_len).to(device)
print(model)
# 모델 초기화
# 임의의 입력 데이터 생성
x = torch.randint(0, vocab, (10, 32)).to(device)  # (배치 크기, 시퀀스 길이)
y = torch.randint(0, vocab, (10, 32)).to(device)
'''
torch.randint(low, high, size): 
low 이상 high 미만의 범위에서 임의의 정수를 가진 텐서를 생성합니다.
여기서 size는 생성할 텐서의 차원을 정의합니다.
'''
# 모델 실행
output = TG(x, y)
print(output.shape) 

cuda mode
TG(
  (embedding): Embedding(10000, 512)
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0-3): 4 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
          )
          (linear1): Linear(in_features=512, out_features=2048, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=2048, out_features=512, bias=True)
          (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
      )
      (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    )
    (decoder): TransformerDecoder(
      (layers): ModuleList(
        (0-3): 4 x TransformerDecoderLayer(
     

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class TD(nn.Module):
    def __init__(self,vocab,d_model,nhead,num_layer,dim_ff,max_len):
        super(TD,self).__init__()
        self.d_model=d_model
        self.embedding=nn.Embedding(vocab,d_model)
        self.pos_encoder=nn.Parameter(torch.randn(1,max_len,d_model))
        self.encoder=nn.TransformerEncoder(nn.TransformerEncoderLayer(d_model=d_model,nhead=nhead,dim_feedforward=dim_ff),num_layers=num_layer)      

        self.fc=nn.Linear(d_model,1)
    def forward(self,x):
        x=self.embedding(x)*torch.sqrt(torch.tensor(self.d_model,dtype=torch.float32))
        x+=self.pos_encoder[:,:x.size(1),:]
        encoder_x=self.encoder(x)
        agg=torch.mean(encoder_x,dim=1)
        output=self.fc(agg)
        return torch.sigmoid(output)
        
device=torch.device('cuda'if torch.cuda.is_available()else'cpu')
print(f'{device} mode')

vocab=10000
d_model=512
nhead=8
num_layer=4
dim_ff=2048
max_len=100

TD=TD(vocab,d_model,nhead,num_layer,dim_ff,max_len).to(device)
#print(model)

x=torch.randint(0,vocab,(10,50)).to(device)
output=TD(x)
print(output)


cuda mode
tensor([[0.4523],
        [0.4711],
        [0.4555],
        [0.4792],
        [0.4638],
        [0.4629],
        [0.4702],
        [0.4274],
        [0.4760],
        [0.4862]], device='cuda:0', grad_fn=<SigmoidBackward0>)


In [14]:
import torch.nn as nn 
import torch.optim as optim
TG=TG.to(device)
TD=TD.to(device)

g_opt=optim.SGD(TG.parameters(),lr=1e-6)
d_opt=optim.SGD(TD.parameters(),lr=1e-6)

criter=nn.BCELoss()#bicrossentropy

num_epochs=1000
batch=64
latent_dim=100

for epoch in range(num_epochs):
    for i, (real_images, _) in enumerate(dataloader):  # 실제 데이터셋에서 이미지 로드
        real_labels = torch.ones(batch_size, 1)
        fake_labels = torch.zeros(batch_size, 1)

        # 실제 이미지를 사용하여 판별자 훈련
        discriminator.zero_grad()
        outputs = discriminator(real_images)
        d_loss_real = criterion(outputs, real_labels)
        d_loss_real.backward()

        # 가짜 이미지 생성 및 판별자 훈련
        noise = torch.randn(batch_size, latent_dim, 1, 1)
        fake_images = generator(noise)
        outputs = discriminator(fake_images.detach())  # 생성자의 그래디언트가 계산되지 않도록 detach() 사용
        d_loss_fake = criterion(outputs, fake_labels)
        d_loss_fake.backward()
        d_optimizer.step()

        # 생성자 훈련
        generator.zero_grad()
        outputs = discriminator(fake_images)
        g_loss = criterion(outputs, real_labels)  # 생성자는 판별자를 속이려고 하므로 실제 라벨 사용
        g_loss.backward()
        g_optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], d_loss: {d_loss_real.item() + d_loss_fake.item()}, g_loss: {g_loss.item()}')

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TransformerGenerator(nn.Module):
    def __init__(self, vocab_size, d_model, nhead, num_encoder_layers, num_decoder_layers, dim_feedforward, max_seq_length):
        super(TransformerGenerator, self).__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoder = nn.Parameter(torch.randn(1, max_seq_length, d_model))
        self.transformer = nn.Transformer(d_model=d_model, nhead=nhead, num_encoder_layers=num_encoder_layers,
                                          num_decoder_layers=num_decoder_layers, dim_feedforward=dim_feedforward)
        self.fc_out = nn.Linear(d_model, vocab_size)
        self.d_model = d_model

    def forward(self, src, tgt):
        src = self.embedding(src) * torch.sqrt(torch.tensor(self.d_model, dtype=torch.float32))
        tgt = self.embedding(tgt) * torch.sqrt(torch.tensor(self.d_model, dtype=torch.float32))
        src += self.pos_encoder[:, :src.size(1), :]
        tgt += self.pos_encoder[:, :tgt.size(1), :]
        
        output = self.transformer(src, tgt)
        output = self.fc_out(output)
        
        return output

# 모델 파라미터 설정
vocab_size = 10000  # 예시로 사용된 어휘 크기
d_model = 512  # 임베딩 차원
nhead = 8  # 멀티헤드 어텐션의 헤드 수
num_encoder_layers = 4  # 인코더 레이어 수
num_decoder_layers = 4  # 디코더 레이어 수
dim_feedforward = 2048  # 피드포워드 네트워크의 차원
max_seq_length = 1000000  # 최대 시퀀스 길이

device=torch.device('cuda'if torch.cuda.is_available()else'cpu')
print(f'{device} mode')

# 모델 초기화
model = TransformerGenerator(vocab_size, d_model, nhead, num_encoder_layers, num_decoder_layers, dim_feedforward, max_seq_length)
model.to(device)
# 임의의 입력 데이터 생성
src = torch.randint(0, vocab_size, (100, 95)).to(device)  # (배치 크기, 시퀀스 길이)
tgt = torch.randint(0, vocab_size, (100, 95)).to(device)

# 모델 실행
output = model(src, tgt)
print(output.shape)  # 출력 크기 확인


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TransformerDiscriminator(nn.Module):
    def __init__(self, vocab_size, d_model, nhead, num_layers, dim_feedforward, max_seq_length):
        super(TransformerDiscriminator, self).__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoder = nn.Parameter(torch.randn(1, max_seq_length, d_model))
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward),
            num_layers=num_layers
        )
        self.fc_out = nn.Linear(d_model, 1)  # 이진 분류를 위한 출력

    def forward(self, src):
        src = self.embedding(src) * torch.sqrt(torch.tensor(d_model, dtype=torch.float32))
        src += self.pos_encoder[:, :src.size(1), :]
        encoded_src = self.transformer_encoder(src)
        # 시퀀스의 각 토큰에 대한 정보를 집계하기 위해 평균을 취합니다.
        aggregated_src = torch.mean(encoded_src, dim=1)
        output = self.fc_out(aggregated_src)
        return torch.sigmoid(output)  # 이진 분류를 위한 시그모이드 활성화 함수

# 모델 파라미터 설정
vocab_size = 10000  # 어휘 크기
d_model = 512  # 임베딩 차원
nhead = 8  # 멀티헤드 어텐션의 헤드 수
num_layers = 4  # 인코더 레이어 수
dim_feedforward = 2048  # 피드포워드 네트워크의 차원
max_seq_length = 100  # 최대 시퀀스 길이

# 모델 초기화
discriminator = TransformerDiscriminator(vocab_size, d_model, nhead, num_layers, dim_feedforward, max_seq_length)

# 임의의 입력 데이터 생성
src = torch.randint(0, vocab_size, (10, 50))  # (배치 크기, 시퀀스 길이)

# 모델 실행
output = discriminator(src)
print(output.shape)  # 출력 크기 확인


In [68]:
import torch
import torch.nn as nn
import torch.optim as optim

class dflks(nn.Module):
    def __init__(self):
        super(dflks,self).__init__()
        self.seq=nn.Sequential(
            nn.Conv2d(10,32,2,stride=1,padding=2),
            nn.Conv2d(32,64,2,stride=1,padding=2),
            nn.MaxPool2d(2,2),
            nn.ReLU(),
            #nn.Flatten(),
            
        )
        #self.fc=nn.Linear(1600,32)
        self.fc=nn.Linear(5,10240)#Flatten 소멸시

    def forward(self,x):
        #print(x.shape)
        x=self.seq(x)
        #print(x.shape)
        x=self.fc(x)
        #print(x.shape)
        return x

cnn = dflks()
output = cnn(torch.randn(32, 10, 4, 4))

print(output.size())

torch.Size([32, 64, 5, 10240])


In [66]:


from PIL import Image
import cv2

img = Image.open("/home/eternal/다운로드/DALL·E 2024-01-21 04.02.17 - A character design of a woman with fox ears, wearing traditional Japanese miko (shrine maiden) attire, featuring a serene and mystical ambiance. The c.png")

print(img.size)


(1024, 1024)
